# Create Tables and Materialized Views

In [8]:
import psycopg2
import json
from getpass import getpass

try:
    with psycopg2.connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database = "UNICEF"
    ) as connection:
        
        create_table_Year_Dimension = """
            CREATE TABLE IF NOT EXISTS public."Year_Dimension"
            (
            "Year_ID" INT NOT NULL,
            "Data_Year" VARCHAR(100),
            CONSTRAINT "Year_ID_pkey" PRIMARY KEY ("Year_ID")
            )
        """
        create_table_Source_Dimension = """
            CREATE TABLE IF NOT EXISTS public."Source_Dimension"
            (
            "Source_ID" INT NOT NULL,
            "Source_Type" VARCHAR(100),
            "Source_Desc" VARCHAR(100),
            CONSTRAINT "Source_ID_pkey" PRIMARY KEY ("Source_ID")
            )
        """
        create_table_Indicator_Dimension = """
            CREATE TABLE IF NOT EXISTS public."Indicator_Dimension"
            (
            "Indicator_ID" INT NOT NULL,
            "Indicator_Type" VARCHAR(100) NOT NULL,
            "Indicator_Desc" VARCHAR(100) NULL,
            CONSTRAINT "Indicator_Dimension_pkey" PRIMARY KEY ("Indicator_ID")
            )
        """  
        create_table_Location_Dimension = """
            CREATE TABLE IF NOT EXISTS public."Location_Dimension"
            (
            "Location_ID" SERIAL NOT NULL,
            "ISOCode" VARCHAR(10) NOT NULL,
            "CountryAndAreaName" VARCHAR(100) NOT NULL,
            "RegionName" VARCHAR(100),
            "DevlopmentStage" VARCHAR(100),
            CONSTRAINT "Location_Dimension_plockey" PRIMARY KEY ("Location_ID")
            )
        """
        create_table_ICT_Skillset_Fact = """
           CREATE TABLE IF NOT EXISTS public."ICT_Skillset_Fact"
            (
            "Skillset_Fact_ID" SERIAL NOT NULL,
            "Location_ID" INT NOT NULL,
            "Indicator_ID" INT NOT NULL,
            "Source_ID" INT NOT NULL,
            "Year_ID" INT NOT NULL,
            "Sex" VARCHAR(10),
            "Moved_File" FLOAT,
            "Copy_Paste" FLOAT,
            "Sent_Email" FLOAT,
            "Used_Excel" FLOAT,
            "Installed_Hardware" FLOAT,
            "Installed_Software" FLOAT,
            "Created_PPT" FLOAT,
            "Transferred_File" FLOAT,
            "Knows_Programming" FLOAT,
            "Atleast_One_Skill_Total" FLOAT,
            "Atleast_One_Skill_Rural" FLOAT,
            "Atleast_One_Skill_Urban" FLOAT,
            "Atleast_One_Skill_Poorest" FLOAT,
            "Atleast_One_Skill_Richest" FLOAT,
            CONSTRAINT "ICT_Skillset_pkey" PRIMARY KEY ("Skillset_Fact_ID"),
            CONSTRAINT "Location_Dimension_ICTfkey" FOREIGN KEY("Location_ID") REFERENCES public."Location_Dimension"("Location_ID") ON UPDATE CASCADE,
            CONSTRAINT "Indicator_ICTfkey" FOREIGN KEY("Indicator_ID") REFERENCES public."Indicator_Dimension"("Indicator_ID") ON UPDATE CASCADE,
            CONSTRAINT "Source_ICTfkey" FOREIGN KEY("Source_ID") REFERENCES public."Source_Dimension"("Source_ID") ON UPDATE CASCADE,
            CONSTRAINT "Year_ICTfkey" FOREIGN KEY("Year_ID") REFERENCES public."Year_Dimension"("Year_ID") ON UPDATE CASCADE
            )
        """   
        create_table_Education_Fact = """
            CREATE TABLE IF NOT EXISTS public."Education_Fact"
            (
            "Education_Fact_ID" SERIAL NOT NULL,
            "Location_ID" INT NOT NULL,
            "Indicator_ID" INT NOT NULL,
            "Source_ID" INT NOT NULL,
            "Year_ID" INT NOT NULL,
            "Total" FLOAT,
            "Female" FLOAT,
            "Male" FLOAT,
            "Urban" FLOAT,
            "Rural" FLOAT,
            "Poorest" FLOAT,
            "Second" FLOAT,
            "Middle" FLOAT,
            "Fourth" FLOAT,
            "Richest" FLOAT,
            CONSTRAINT "Education_Fact_pkey" PRIMARY KEY ("Education_Fact_ID"),
            CONSTRAINT "Location_Dimension_EDfkey" FOREIGN KEY("Location_ID") REFERENCES public."Location_Dimension"("Location_ID") ON UPDATE CASCADE,
            CONSTRAINT "Indicator_EDfkey" FOREIGN KEY("Indicator_ID") REFERENCES public."Indicator_Dimension"("Indicator_ID") ON UPDATE CASCADE,
            CONSTRAINT "Source_EDfkey" FOREIGN KEY("Source_ID") REFERENCES public."Source_Dimension"("Source_ID") ON UPDATE CASCADE,
            CONSTRAINT "Year_EDfkey" FOREIGN KEY("Year_ID") REFERENCES public."Year_Dimension"("Year_ID") ON UPDATE CASCADE
            )
        """ 
        create_table_Death_Mortality_WASH_Fact = """
            CREATE TABLE IF NOT EXISTS public."Death_Mortality_WASH_Fact"
            (
            "Fact_ID" SERIAL NOT NULL,
            "Location_ID" INT NOT NULL,
            "Indicator_ID" INT NOT NULL,
            "Source_ID" INT NOT NULL,
            "Year_ID" INT NOT NULL,
            "Obs_Value" FLOAT,
            CONSTRAINT "Fact_ID_pkey" PRIMARY KEY ("Fact_ID"),
            CONSTRAINT "Location_Dimension_DMfkey" FOREIGN KEY("Location_ID") REFERENCES public."Location_Dimension"("Location_ID") ON UPDATE CASCADE,
            CONSTRAINT "Indicator_BMfkey" FOREIGN KEY("Indicator_ID") REFERENCES public."Indicator_Dimension"("Indicator_ID") ON UPDATE CASCADE,
            CONSTRAINT "Source_DMfkey" FOREIGN KEY("Source_ID") REFERENCES public."Source_Dimension"("Source_ID") ON UPDATE CASCADE,
            CONSTRAINT "Year_DMfkey" FOREIGN KEY("Year_ID") REFERENCES public."Year_Dimension"("Year_ID") ON UPDATE CASCADE
            )
        """ 
        create_table_Death_Mortality_WASH_Fact_View = """
            CREATE MATERIALIZED VIEW  IF NOT EXISTS Death_Mortality_WASH_Fact_View AS 
            select "Death_Mortality_WASH_Fact".*,"Location_Dimension"."ISOCode", "Location_Dimension"."CountryAndAreaName","Location_Dimension"."RegionName","Location_Dimension"."DevlopmentStage",
            "Indicator_Dimension"."Indicator_Type","Year_Dimension"."Data_Year","Source_Dimension"."Source_Type","Source_Dimension"."Source_Desc"
            from public."Death_Mortality_WASH_Fact",public."Location_Dimension",public."Indicator_Dimension",public."Year_Dimension",public."Source_Dimension"
            where  "Death_Mortality_WASH_Fact"."Location_ID" = "Location_Dimension"."Location_ID"
            and "Death_Mortality_WASH_Fact"."Year_ID" = "Year_Dimension"."Year_ID"
            and "Death_Mortality_WASH_Fact"."Source_ID" = "Source_Dimension"."Source_ID"
            and "Death_Mortality_WASH_Fact"."Indicator_ID" = "Indicator_Dimension"."Indicator_ID"
        """ 
        create_table_ICT_Skillset_Fact_View = """
            CREATE MATERIALIZED VIEW  IF NOT EXISTS ICT_Skillset_Fact_View AS 
            select "ICT_Skillset_Fact".*,"Location_Dimension"."ISOCode", "Location_Dimension"."CountryAndAreaName","Location_Dimension"."RegionName","Location_Dimension"."DevlopmentStage",
            "Indicator_Dimension"."Indicator_Type","Year_Dimension"."Data_Year","Source_Dimension"."Source_Type","Source_Dimension"."Source_Desc"
            from public."ICT_Skillset_Fact",public."Location_Dimension",public."Indicator_Dimension",public."Year_Dimension",public."Source_Dimension"
            where  "ICT_Skillset_Fact"."Location_ID" = "Location_Dimension"."Location_ID"
            and "ICT_Skillset_Fact"."Year_ID" = "Year_Dimension"."Year_ID"
            and "ICT_Skillset_Fact"."Source_ID" = "Source_Dimension"."Source_ID"
            and "ICT_Skillset_Fact"."Indicator_ID" = "Indicator_Dimension"."Indicator_ID"
        """ 
        create_table_Education_Fact_View = """
           CREATE MATERIALIZED VIEW  IF NOT EXISTS Education_Fact_View AS 
           select "Education_Fact".*,"Location_Dimension"."ISOCode", "Location_Dimension"."CountryAndAreaName","Location_Dimension"."RegionName","Location_Dimension"."DevlopmentStage",
           "Indicator_Dimension"."Indicator_Type","Year_Dimension"."Data_Year","Source_Dimension"."Source_Type","Source_Dimension"."Source_Desc"
           from public."Education_Fact",public."Location_Dimension",public."Indicator_Dimension",public."Year_Dimension",public."Source_Dimension"
           where  "Education_Fact"."Location_ID" = "Location_Dimension"."Location_ID"
           and "Education_Fact"."Year_ID" = "Year_Dimension"."Year_ID"
           and "Education_Fact"."Source_ID" = "Source_Dimension"."Source_ID"
           and "Education_Fact"."Indicator_ID" = "Indicator_Dimension"."Indicator_ID"
        """ 
        with connection.cursor() as cursor:
            cursor.execute(create_table_Year_Dimension)
            cursor.execute(create_table_Source_Dimension)
            cursor.execute(create_table_Indicator_Dimension)
            cursor.execute(create_table_Location_Dimension)
            cursor.execute(create_table_ICT_Skillset_Fact)
            cursor.execute(create_table_Education_Fact)
            cursor.execute(create_table_Death_Mortality_WASH_Fact)
            cursor.execute(create_table_Death_Mortality_WASH_Fact_View)
            cursor.execute(create_table_ICT_Skillset_Fact_View)
            cursor.execute(create_table_Education_Fact_View)
            connection.commit()
            print("DDL Executed Successfully")
except BaseException as e:
    connection.rollback()
    print("DDL Execution Aborted")
    print(e)
    
connection.close()

Enter username: postgres
Enter password: ········
DDL Executed Successfully


# Insert Rows from CSV into Dimension Tables

In [27]:
import psycopg2
import json
from getpass import getpass
import pandas as pnd
import os

try:
    # Import Indicator Dimension CSV files into dataframe
    data_Indicator = pnd.read_csv(os.getcwd()+'/Indicator_Dimension.csv')
    df_Indicator = pnd.DataFrame(data_Indicator)
    with psycopg2.connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database = "UNICEF"
    ) as connection:
        for row_loc in df_Indicator.itertuples():
            with connection.cursor() as cursor:
                add_loc = ("INSERT INTO public.\"Indicator_Dimension\" VALUES (%s,%s,%s)")
                data_loc = (row_loc.Indicator_ID, row_loc.Indicator_Type, row_loc.Indicator_Desc)
                cursor.execute(add_loc, data_loc)
                connection.commit()  
        print("Indicator Dimension Table rows added successfully")
        
        data_Year = pnd.read_csv(os.getcwd()+'/Year_Dimension.csv')
        df_Year = pnd.DataFrame(data_Year)
        for row_loc in df_Year.itertuples():
            with connection.cursor() as cursor:
                add_loc = ("INSERT INTO public.\"Year_Dimension\" VALUES (%s,%s)")
                data_loc = (row_loc.Year_ID, row_loc.Data_Year)
                cursor.execute(add_loc, data_loc)
                connection.commit()  
        print("Year Dimension Table rows added successfully")
        
        data_Source = pnd.read_csv(os.getcwd()+'/Sourc_Dimension.csv')
        df_Source = pnd.DataFrame(data_Source)
        for row_loc in df_Source.itertuples():
            with connection.cursor() as cursor:
                add_loc = ("INSERT INTO public.\"Source_Dimension\" VALUES (%s,%s,%s)")
                data_loc = (row_loc.ID, row_loc.Source_Type, row_loc.Desc)
                cursor.execute(add_loc, data_loc)
                connection.commit()  
        print("Source Dimension Table rows added successfully")
        
except BaseException as e:
    connection.rollback()
    print(e)
    
connection.close()

Enter username: postgres
Enter password: ········
Indicator Dimension Table rows added successfully
Year Dimension Table rows added successfully
Source Dimension Table rows added successfully


# Export Data into CSV files to use in Tableau

In [28]:
import csv
import os
import psycopg2
from getpass import getpass

# Define CSV File path and Name
CSVOutputfilePath = os.getcwd()+'/'
CSVOutputfileName = 'Death_And_WASH.csv'
connect_postgres = None

# If the file path exists then
if os.path.exists(CSVOutputfilePath):
    try:
        # Connect to postgres UNICEF database.
        connect_postgres = psycopg2.connect(
        host="localhost",
        user= input("Enter username: "),
        password= getpass("Enter password: "),
        database = "UNICEF"
        )

    except psycopg2.DatabaseError as e:
        print("Connected to Database.")
        quit()

    data_cursor = connect_postgres.cursor()

    # Select Death_Mortality_WASH_Fact with rollup and grouping identifiers.
    sqlSelectData = """select "Death_Mortality_WASH_Fact".*,"Location_Dimension"."ISOCode", "Location_Dimension"."CountryAndAreaName","Location_Dimension"."RegionName","Location_Dimension"."DevlopmentStage",
                    "Indicator_Dimension"."Indicator_Type","Year_Dimension"."Data_Year","Source_Dimension"."Source_Type","Source_Dimension"."Source_Desc"
                    from public."Death_Mortality_WASH_Fact",public."Location_Dimension",public."Indicator_Dimension",public."Year_Dimension",public."Source_Dimension"
                    where  "Death_Mortality_WASH_Fact"."Location_ID" = "Location_Dimension"."Location_ID"
                    and "Death_Mortality_WASH_Fact"."Year_ID" = "Year_Dimension"."Year_ID"
                    and "Death_Mortality_WASH_Fact"."Source_ID" = "Source_Dimension"."Source_ID"
                    and "Death_Mortality_WASH_Fact"."Indicator_ID" = "Indicator_Dimension"."Indicator_ID"
                    """   
    try:
        data_cursor.execute(sqlSelectData)
        resultData = data_cursor.fetchall()
        headers_Row = [row_num[0] for row_num in data_cursor.description]

        # Open the output CSV file for writing.
        OutputcsvFile = csv.writer(open(CSVOutputfilePath + CSVOutputfileName, 'w', newline=''),
                             delimiter=',', lineterminator='\r\n',
                             quoting=csv.QUOTE_ALL, escapechar='\\')

        # Adding header row and SQL data to the CSV file.
        OutputcsvFile.writerow(headers_Row)
        OutputcsvFile.writerows(resultData)
        #print(resultData)
        print("File created successfully.")

    except psycopg2.DatabaseError as e:
        print("File creation unsuccessful.")
        quit()

    finally:
        connect_postgres.close()

else:
    print("File creation unsuccessful.")

Enter username: postgres
Enter password: ········
File created successfully.


In [3]:
import csv
import os
import psycopg2
from getpass import getpass

# Define CSV File path and Name
CSVOutputfilePath = os.getcwd()+'/'
CSVOutputfileName = 'ICT_Skillset.csv'
connect_postgres = None

# If the file path exists then
if os.path.exists(CSVOutputfilePath):
    try:
        # Connect to postgres UNICEF database.
        connect_postgres = psycopg2.connect(
        host="localhost",
        user= input("Enter username: "),
        password= getpass("Enter password: "),
        database = "UNICEF"
        )

    except psycopg2.DatabaseError as e:
        print("Connected to Database.")
        quit()

    data_cursor = connect_postgres.cursor()

    # Select Death_Mortality_WASH_Fact with rollup and grouping identifiers.
    sqlSelectData = """select "ICT_Skillset_Fact".*,"Location_Dimension"."ISOCode", "Location_Dimension"."CountryAndAreaName","Location_Dimension"."RegionName","Location_Dimension"."DevlopmentStage",
                    "Indicator_Dimension"."Indicator_Type","Year_Dimension"."Data_Year","Source_Dimension"."Source_Type","Source_Dimension"."Source_Desc"
                    from public."ICT_Skillset_Fact",public."Location_Dimension",public."Indicator_Dimension",public."Year_Dimension",public."Source_Dimension"
                    where  "ICT_Skillset_Fact"."Location_ID" = "Location_Dimension"."Location_ID"
                    and "ICT_Skillset_Fact"."Year_ID" = "Year_Dimension"."Year_ID"
                    and "ICT_Skillset_Fact"."Source_ID" = "Source_Dimension"."Source_ID"
                    and "ICT_Skillset_Fact"."Indicator_ID" = "Indicator_Dimension"."Indicator_ID"
                    """   
    try:
        data_cursor.execute(sqlSelectData)
        resultData = data_cursor.fetchall()
        headers_Row = [row_num[0] for row_num in data_cursor.description]

        # Open the output CSV file for writing.
        OutputcsvFile = csv.writer(open(CSVOutputfilePath + CSVOutputfileName, 'w', newline=''),
                             delimiter=',', lineterminator='\r\n',
                             quoting=csv.QUOTE_ALL, escapechar='\\')

        # Adding header row and SQL data to the CSV file.
        OutputcsvFile.writerow(headers_Row)
        OutputcsvFile.writerows(resultData)
        #print(resultData)
        print("File created successfully.")

    except psycopg2.DatabaseError as e:
        print("File creation unsuccessful.")
        quit()

    finally:
        connect_postgres.close()

else:
    print("File creation unsuccessful.")

Enter username: postgres
Enter password: ········
File created successfully.


In [4]:
import csv
import os
import psycopg2
from getpass import getpass

# Define CSV File path and Name
CSVOutputfilePath = os.getcwd()+'/'
CSVOutputfileName = 'Education_Facts.csv'
connect_postgres = None

# If the file path exists then
if os.path.exists(CSVOutputfilePath):
    try:
        # Connect to postgres UNICEF database.
        connect_postgres = psycopg2.connect(
        host="localhost",
        user= input("Enter username: "),
        password= getpass("Enter password: "),
        database = "UNICEF"
        )

    except psycopg2.DatabaseError as e:
        print("Connected to Database.")
        quit()

    data_cursor = connect_postgres.cursor()

    # Select Death_Mortality_WASH_Fact with rollup and grouping identifiers.
    sqlSelectData = """select "Education_Fact".*,"Location_Dimension"."ISOCode", "Location_Dimension"."CountryAndAreaName","Location_Dimension"."RegionName","Location_Dimension"."DevlopmentStage",
                    "Indicator_Dimension"."Indicator_Type","Year_Dimension"."Data_Year","Source_Dimension"."Source_Type","Source_Dimension"."Source_Desc"
                    from public."Education_Fact",public."Location_Dimension",public."Indicator_Dimension",public."Year_Dimension",public."Source_Dimension"
                    where  "Education_Fact"."Location_ID" = "Location_Dimension"."Location_ID"
                    and "Education_Fact"."Year_ID" = "Year_Dimension"."Year_ID"
                    and "Education_Fact"."Source_ID" = "Source_Dimension"."Source_ID"
                    and "Education_Fact"."Indicator_ID" = "Indicator_Dimension"."Indicator_ID"
                    """   
    try:
        data_cursor.execute(sqlSelectData)
        resultData = data_cursor.fetchall()
        headers_Row = [row_num[0] for row_num in data_cursor.description]

        # Open the output CSV file for writing.
        OutputcsvFile = csv.writer(open(CSVOutputfilePath + CSVOutputfileName, 'w', newline=''),
                             delimiter=',', lineterminator='\r\n',
                             quoting=csv.QUOTE_ALL, escapechar='\\')

        # Adding header row and SQL data to the CSV file.
        OutputcsvFile.writerow(headers_Row)
        OutputcsvFile.writerows(resultData)
        #print(resultData)
        print("File created successfully.")

    except psycopg2.DatabaseError as e:
        print("File creation unsuccessful.")
        quit()

    finally:
        connect_postgres.close()

else:
    print("File creation unsuccessful.")

Enter username: postgres
Enter password: ········
File created successfully.
